In [ ]:
# 2. Завантаження даних (вибрав CIFAR-10)
# а) Завантаження датасету
!pip install tensorflow

In [ ]:
# б) обробка даних
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Завантаження датасету
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('Train data shape:', x_train.shape)
print('Test data shape:', x_test.shape)

# Масштабування зображень до діапазону [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Зберігання оброблених даних
import numpy as np
np.save('x_train.npy', x_train)
np.save('y_train.npy', y_train)
np.save('x_test.npy', x_test)
np.save('y_test.npy', y_test)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Train data shape: (50000, 32, 32, 3)
Test data shape: (10000, 32, 32, 3)


In [ ]:
# c) завантаження даних у Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp x_train.npy /content/drive/MyDrive/AML_lab16/
!cp y_train.npy /content/drive/MyDrive/AML_lab16/
!cp x_test.npy /content/drive/MyDrive/AML_lab16/
!cp y_test.npy /content/drive/MyDrive/AML_lab16/

Mounted at /content/drive


In [ ]:
# 3. Розробка базової моделі
# a) створення CNN моделі
from tensorflow.keras import layers, models

# Визначення моделі
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 класів для CIFAR-10
])

# Компіляція моделі
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Виведення архітектури моделі
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 4, 4, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          65,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 122,570 (478.79 KB)

 Trainable params: 122,570 (478.79 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# б) навчання моделі
import numpy as np

# Вибір 5% даних
num_samples = int(0.05 * x_train.shape[0])  # 5% від кількості тренувальних зразків
x_train_small = x_train[:num_samples]
y_train_small = y_train[:num_samples]

# Навчання моделі
history = model.fit(x_train_small, y_train_small, epochs=5, batch_size=64, validation_split=0.2)

# Оцінка моделі на тестових даних
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 164ms/step - accuracy: 0.7637 - loss: 0.6680 - val_accuracy: 0.4380 - val_loss: 1.8095
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 87ms/step - accuracy: 0.8041 - loss: 0.6092 - val_accuracy: 0.4500 - val_loss: 1.8161
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 162ms/step - accuracy: 0.8192 - loss: 0.5463 - val_accuracy: 0.4480 - val_loss: 1.8739
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - accuracy: 0.8321 - loss: 0.5221 - val_accuracy: 0.4740 - val_loss: 1.9184
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - accuracy: 0.8525 - loss: 0.4434 - val_accuracy: 0.4700 - val_loss: 1.9841
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.4709 - loss: 1.9329
Test accuracy: 0.4717999994754791


In [ ]:
# 4. Реалізація стратегій активного навчання
# a) реалізація Uncertainty Sampling та Random Sampling
import random

# Стратегія Random Sampling (випадковий вибір зразків)
def random_sampling(unlabelled_data, sample_size):
    indices = random.sample(range(len(unlabelled_data)), sample_size)
    return indices

# Стратегія Uncertainty Sampling (вибір зразків з найбільшою невизначеністю)
def uncertainty_sampling(model, unlabelled_data, sample_size):
    # Прогнозуємо ймовірності на невідомих даних
    predictions = model.predict(unlabelled_data)

    # Визначаємо невизначеність як зразки з мінімальною впевненістю у прогнозі
    uncertainty = np.max(predictions, axis=1)

    # Вибираємо зразки з найвищою невизначеністю
    uncertain_indices = np.argsort(uncertainty)[:sample_size]

    return uncertain_indices

In [ ]:
# б) створення функцій оновлення набору даних та перенавчання моделі

# Оновлення тренувального набору після вибору нових зразків
def update_dataset(labeled_data, labeled_labels, unlabelled_data, unlabelled_labels, selected_indices):
    new_data = unlabelled_data[selected_indices]
    new_labels = unlabelled_labels[selected_indices]

    # Додаємо нові зразки до вже наявного тренувального набору
    labeled_data = np.concatenate([labeled_data, new_data])
    labeled_labels = np.concatenate([labeled_labels, new_labels])

    # Видаляємо додані зразки з невідомих
    unlabelled_data = np.delete(unlabelled_data, selected_indices, axis=0)
    unlabelled_labels = np.delete(unlabelled_labels, selected_indices, axis=0)

    return labeled_data, labeled_labels, unlabelled_data, unlabelled_labels

# Функція для перенавчання моделі
def retrain_model(model, labeled_data, labeled_labels):
    model.fit(labeled_data, labeled_labels, epochs=5, batch_size=64, validation_split=0.2)

In [ ]:
# 5. Розробка конвеєрів активного навчання
# Конвеєр 1: Ручний вибір зразків, локальне зберігання даних, ручне навчання моделі

# Зберігання даних локально
def save_locally(labeled_data, labeled_labels, iteration):
    np.save(f'x_train_local_{iteration}.npy', labeled_data)
    np.save(f'y_train_local_{iteration}.npy', labeled_labels)

# Конвеєр 1 (локальне зберігання)
def pipeline_1(model, labeled_data, labeled_labels, unlabelled_data, unlabelled_labels, iteration):
    # Ручний вибір зразків
    selected_indices = random_sampling(unlabelled_data, sample_size=100)

    # Оновлення набору даних
    labeled_data, labeled_labels, unlabelled_data, unlabelled_labels = update_dataset(
        labeled_data, labeled_labels, unlabelled_data, unlabelled_labels, selected_indices
    )

    # Перенавчання моделі
    retrain_model(model, labeled_data, labeled_labels)

    # Локальне зберігання даних
    save_locally(labeled_data, labeled_labels, iteration)

In [ ]:
# Функція для зберігання результатів обчислень в гугл диск
# Конвеєр 2: Автоматичний вибір зразків (Uncertainty Sampling), Google Drive для зберігання, напівавтоматичне навчання

# Зберігання даних на Google Drive
def save_to_drive(labeled_data, labeled_labels, iteration):
    np.save(f'/content/drive/MyDrive/AML_lab16/x_train_drive_{iteration}.npy', labeled_data)
    np.save(f'/content/drive/MyDrive/AML_lab16/y_train_drive_{iteration}.npy', labeled_labels)

# Конвеєр 2
def pipeline_2(model, labeled_data, labeled_labels, unlabelled_data, unlabelled_labels, iteration):
    # Автоматичний вибір зразків (Uncertainty Sampling)
    selected_indices = uncertainty_sampling(model, unlabelled_data, sample_size=100)

    # Оновлення набору даних
    labeled_data, labeled_labels, unlabelled_data, unlabelled_labels = update_dataset(
        labeled_data, labeled_labels, unlabelled_data, unlabelled_labels, selected_indices
    )

    # Перенавчання моделі
    retrain_model(model, labeled_data, labeled_labels)

    # Зберігання на Google Drive
    save_to_drive(labeled_data, labeled_labels, iteration)

In [ ]:
# Конвеєр 3: Автоматизовний

# Зберігання даних на Google Drive та використання Colab API
def save_to_drive_colab(labeled_data, labeled_labels, iteration):
    # Зберігаємо на Google Drive
    np.save(f'/content/drive/MyDrive/AML_lab16/x_train_colab_{iteration}.npy', labeled_data)
    np.save(f'/content/drive/MyDrive/AML_lab16/y_train_colab_{iteration}.npy', labeled_labels)

    print(f"Дані збережені на Google Drive. Використовується Colab для перенавчання на ітерації {iteration}.")

# Конвеєр 3 (Google Drive + Colab)
def pipeline_3(model, labeled_data, labeled_labels, unlabelled_data, unlabelled_labels, iteration):
    # Вибір стратегії (Uncertainty або Random Sampling)
    if iteration % 2 == 0:
        selected_indices = uncertainty_sampling(model, unlabelled_data, sample_size=100)
    else:
        selected_indices = random_sampling(unlabelled_data, sample_size=100)

    # Оновлення набору даних
    labeled_data, labeled_labels, unlabelled_data, unlabelled_labels = update_dataset(
        labeled_data, labeled_labels, unlabelled_data, unlabelled_labels, selected_indices
    )

    # Автоматичне навчання моделі через API
    retrain_model(model, labeled_data, labeled_labels)

    # Зберігання на Google Drive та Colab API
    save_to_drive_colab(labeled_data, labeled_labels, iteration)

In [ ]:
import pandas as pd
import gspread
import time
import psutil
from google.colab import auth
from google.auth import default

# Авторизація користувача
auth.authenticate_user()
creds, _ = default()
gs = gspread.authorize(creds)

# Підключення до Google Sheets
def connect_to_google_sheets_by_id(sheet_id):
    return gs.open_by_key(sheet_id).sheet1

# ID Google Sheets
sheet_id = '1dt6cB8Xh_y7QR-zW3RwfwSSEcxZXP3UE4OwPP_9DJ5g'
sheet = connect_to_google_sheets_by_id(sheet_id)

# Функція для запису імен стовпців у Google Sheets
def log_headers_to_sheet(sheet):
    headers = ["Pipeline", "Iteration", "Test Accuracy", "CPU Usage (%)", "Memory Usage (%)", "Elapsed Time (s)"]
    sheet.append_row(headers)

# Функція для запису прогресу в Google Sheets
def log_progress_to_sheet(sheet, pipeline_name, iteration, test_acc, cpu_usage, memory_usage, elapsed_time):
    data = [pipeline_name, iteration + 1, test_acc, cpu_usage, memory_usage, elapsed_time]
    sheet.append_row(data)

# Основний цикл активного навчання з відстеженням прогресу в Google Sheets
def active_learning_cycle_with_logging(pipeline, pipeline_name, model, x_train, y_train, x_test, y_test, save_method, sheet):
    labeled_data = x_train[:num_samples]  # 5% початкових даних
    labeled_labels = y_train[:num_samples]
    unlabelled_data = x_train[num_samples:]  # решта даних
    unlabelled_labels = y_train[num_samples:]

    total_time = 0  # Змінна для відстеження загального часу

    # Логування заголовків у Google Sheets (один раз перед циклом)
    log_headers_to_sheet(sheet)

    for iteration in range(5):
        print(f"Iteration {iteration + 1}")

        # Вимір часу та ресурсів перед ітерацією
        start_time = time.time()
        cpu_usage_before = psutil.cpu_percent()
        memory_usage_before = psutil.virtual_memory().percent

        # Виклик обраного конвеєра
        pipeline(model, labeled_data, labeled_labels, unlabelled_data, unlabelled_labels, iteration)

        # Оцінка моделі
        test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
        print(f"Test accuracy after iteration {iteration + 1}: {test_acc}")

        # Вимір часу та ресурсів після ітерації
        elapsed_time = time.time() - start_time
        total_time += elapsed_time  # Додаємо час ітерації до загального часу
        cpu_usage_after = psutil.cpu_percent()
        memory_usage_after = psutil.virtual_memory().percent

        # Обчислення середнього використання ресурсів
        cpu_usage = (cpu_usage_before + cpu_usage_after) / 2
        memory_usage = (memory_usage_before + memory_usage_after) / 2

        # Логування результатів в Google Sheets (додаємо назву конвеєра)
        log_progress_to_sheet(sheet, pipeline_name, iteration, test_acc, cpu_usage, memory_usage, elapsed_time)

    return total_time  # Повертаємо загальний час для оцінки

# Оголошуємо кількість вибраних даних для першого навчання
num_samples = int(0.05 * x_train.shape[0])  # 5% від кількості тренувальних зразків

# Функція для оцінки ефективності кожного конвеєра
def evaluate_pipeline(pipeline, pipeline_name, model, x_train, y_train, x_test, y_test, save_method, sheet):
    total_time = active_learning_cycle_with_logging(pipeline, pipeline_name, model, x_train, y_train, x_test, y_test, save_method, sheet)
    iterations_per_hour = 5 / (total_time / 3600)  # Обчислення кількості ітерацій за годину

    print(f"Total time for pipeline {pipeline_name}: {total_time} seconds")
    print(f"Iterations per hour for pipeline {pipeline_name}: {iterations_per_hour}")

    # Для масштабованості — симуляція на збільшеному обсязі даних
    num_samples_large = int(0.15 * x_train.shape[0])  # 15% для тестування масштабованості
    x_train_large = x_train[:num_samples_large]
    y_train_large = y_train[:num_samples_large]

    # Можна викликати той самий конвеєр для тестування масштабованості:
    print(f"Testing scalability for pipeline {pipeline_name}...")
    total_time_large = active_learning_cycle_with_logging(pipeline, pipeline_name, model, x_train_large, y_train_large, x_test, y_test, save_method, sheet)

    # Результати масштабованості
    iterations_per_hour_large = 5 / (total_time_large / 3600)
    print(f"Iterations per hour with larger dataset for {pipeline_name}: {iterations_per_hour_large}")

# Використання циклу з відстеженням прогресу в Google Sheets

# Конвеєр 1 - локальне зберігання
evaluate_pipeline(pipeline_1, "Pipeline 1 - Local", model, x_train, y_train, x_test, y_test, save_method="local", sheet=sheet)

# Конвеєр 2 - Google Drive
evaluate_pipeline(pipeline_2, "Pipeline 2 - Google Drive", model, x_train, y_train, x_test, y_test, save_method="drive", sheet=sheet)

# Конвеєр 3 - Google Drive + Colab API
evaluate_pipeline(pipeline_3, "Pipeline 3 - Google Drive + Colab", model, x_train, y_train, x_test, y_test, save_method="colab", sheet=sheet)

Iteration 1
Epoch 1/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 159ms/step - accuracy: 0.8777 - loss: 0.4378 - val_accuracy: 0.4769 - val_loss: 1.9353
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - accuracy: 0.8765 - loss: 0.4287 - val_accuracy: 0.5000 - val_loss: 1.9174
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.8828 - loss: 0.3993 - val_accuracy: 0.4885 - val_loss: 1.8912
Epoch 4/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.9232 - loss: 0.3166 - val_accuracy: 0.4750 - val_loss: 2.0621
Epoch 5/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 141ms/step - accuracy: 0.9306 - loss: 0.2833 - val_accuracy: 0.4769 - val_loss: 2.0583
Test accuracy after iteration 1: 0.4528000056743622
Iteration 2
Epoch 1/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 158ms/step - accuracy: 0.9465 - loss: 0.2429 - val_accuracy: 0.5115 - val_loss: 2.1620
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 167ms/step - accuracy: 0.9536 - loss: 0.2205 - val_accuracy: 0.4865 - val_loss: 2.3749
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 8